In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Mon Mar 1 2018

@author: Ray


'''

#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''
import warnings
warnings.filterwarnings("ignore")
from glob import glob
import gc
import os
from tqdm import tqdm
import time
from itertools import chain
import pandas as pd
import numpy as np
from xgboost import plot_importance
from xgboost import XGBClassifier
#from matplotlib import pyplot
import utils # made by author for efficiently dealing with data
import pickle

In [12]:
###############################
# setting
###############################

DATE = '0308'
utils.mkdir_p('../output/model/{}/'.format(DATE))
utils.mkdir_p('../output/sub/{}/'.format(DATE))

# using different random seed to make sure variety of models
seed = 72
np.random.seed(seed) 

print("""#==== print param ======""")
print('DATE:', DATE)
print('seed:', seed)


##########################################
#load dataset
##########################################

# load dataset
file_name = '../output/model/xgb_feature_tuning_seed_72.model'
train_0 = utils.load_pred_feature('trainW-0', keep_all = False, model_file_name = file_name, n_top_features = 48)
train_1 = utils.load_pred_feature('trainW-1', keep_all = False, model_file_name = file_name, n_top_features = 48)
train_2 = utils.load_pred_feature('trainW-2', keep_all = False, model_file_name = file_name, n_top_features = 48)
# make data augmentation having same label distribution with training set provided by the kkbox
per_churned_in_train_0 = train_0[['is_churn']].describe().ix['mean'][0] 
n_churned = train_1[train_1.is_churn == 0].shape[0] * per_churned_in_train_0
print('per_churned_in_train_0', per_churned_in_train_0)
print('n_churned', int(n_churned))
train_1 = pd.concat([train_1[train_1.is_churn == 0],
                train_1[train_1.is_churn == 1].sample(n = int(n_churned), random_state = seed)
               ], ignore_index=True)
per_churned_in_train_1 = train_1[['is_churn']].describe().ix['mean'][0] 
print('per_churned_in_train_1', per_churned_in_train_1)
train_2 = pd.concat([train_2[train_2.is_churn == 0],
                train_2[train_2.is_churn == 1].sample(n = int(n_churned), random_state = seed)
               ], ignore_index=True)
per_churned_in_train_2 = train_1[['is_churn']].describe().ix['mean'][0] 
print('per_churned_in_train_2', per_churned_in_train_2)
train = pd.concat([train_0, train_1, train_2], ignore_index=True)

# reduce memory in python
del train_0, train_1, train_2, per_churned_in_train_0, per_churned_in_train_1, per_churned_in_train_2, n_churned,
gc.collect()


#==============================================================================
# prepare training data
#==============================================================================
X_train = train.drop('is_churn', axis=1)
train_user = X_train[['msno']].drop_duplicates()
Y_train = train['is_churn']
del train
gc.collect()
print ('prepartion of training set is done')


  0%|          | 0/20 [00:00<?, ?it/s]

#==== print param ======
DATE: 0308
seed: 72
keep top imp
True


  0%|          | 0/20 [00:00<?, ?it/s]

trainW-0.shape:(881896, 50)

keep top imp
True


  0%|          | 0/20 [00:00<?, ?it/s]

trainW-1.shape:(884309, 50)

keep top imp
True


100%|██████████| 20/20 [01:56<00:00,  5.83s/it]


trainW-2.shape:(905872, 50)

per_churned_in_train_0 0.0587484238504
n_churned 10042
per_churned_in_train_1 0.0554877139084
per_churned_in_train_2 0.0554877139084
prepartion of training set is done


In [37]:
def split_build_valid(valid_size = 0.05):
    '''
    Hold-out validation

    parameters:
    ------------------
    valid_size: float

    '''
    # train/val split by user
    train_user['is_valid'] = np.random.choice([0,1], size=len(train_user), 
                                              p=[1-valid_size, valid_size]) # randomly pick someone as validation user
    # is_valid: 1 if the user is validating user else 0
    valid_n = train_user['is_valid'].sum()
    build_n = (train_user.shape[0] - valid_n)
    print('build user:{}, valid user:{}'.format(build_n, valid_n))
    valid_user = train_user[train_user['is_valid']==1].msno
    is_valid = X_train.msno.isin(valid_user)
    # to create the XGBoost matrices that will be used to train the model using XGBoost. 
    x_train = X_train[~is_valid].drop('msno', axis=1)
    y_train = Y_train[~is_valid]
    x_val = X_train[is_valid].drop('msno', axis=1)
    y_val = Y_train[is_valid]
    print('FINAL SHAPE')
    print('x_train.shape:{0}'.format(x_train.shape))
    print('x_val.shape:{0}'.format(x_val.shape))

    return x_train, y_train, x_val, y_val




# xgboost parameters (tuning result from cv)
params_fixed = {
	'objective' : 'binary:logistic',
	'max_depth': 7,
	'min_child_weight': 4,
	'gamma': 0.0,
	'subsample' : 0.95,
	'colsample_bytree': 0.75,
	'reg_lambda': 5,
	'seed': seed
}

#Here we did is lower the learning rate and add more trees.
n_estimators = 10000
learning_rate = 0.01
early_stopping_rounds = 50

# for simple ensemble
LOOP = 2

# Core
models = [] # for the following prediction
for i in range(LOOP):
    print('LOOP',i)
    # hold-out validation
    x_train, y_train, x_val, y_val = split_build_valid(valid_size = 0.05)
    # model training
    model = XGBClassifier(
        **params_fixed,
        n_estimators = n_estimators,
        learning_rate = learning_rate
                             )
    model.fit(x_train, y_train, 
                  eval_metric ='logloss' , eval_set = [(x_val,y_val)],
                  early_stopping_rounds = early_stopping_rounds) 
    # saving
    models.append(model)
    pickle.dump(model, open('../output/model/{}/xgb_churn_{}.model'.format(DATE, i), "wb"))
    # validating
    valid_yhat = model.predict(x_val) # y_hat is result of prediction
    print('Valid Mean:', np.mean(valid_yhat))
    del x_train, y_train, x_val, y_val
    gc.collect()


LOOP 0
build user:865155, valid user:45317
FINAL SHAPE
x_train.shape:(1074682, 48)
x_val.shape:(56284, 48)
[0]	validation_0-logloss:0.684692
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.676399
[2]	validation_0-logloss:0.66828
[3]	validation_0-logloss:0.660305
[4]	validation_0-logloss:0.652473
[5]	validation_0-logloss:0.64479
[6]	validation_0-logloss:0.637259
[7]	validation_0-logloss:0.629933
[8]	validation_0-logloss:0.622681
[9]	validation_0-logloss:0.615573
[10]	validation_0-logloss:0.608633
[11]	validation_0-logloss:0.60177
[12]	validation_0-logloss:0.595035
[13]	validation_0-logloss:0.588423
[14]	validation_0-logloss:0.581918
[15]	validation_0-logloss:0.57554
[16]	validation_0-logloss:0.569348
[17]	validation_0-logloss:0.56318
[18]	validation_0-logloss:0.557164
[19]	validation_0-logloss:0.551271
[20]	validation_0-logloss:0.545415
[21]	validation_0-logloss:0.539669
[22]	validation_0-logloss:0.534014
[23]	validation_0-logloss:0.528454


[227]	validation_0-logloss:0.165588
[228]	validation_0-logloss:0.16521
[229]	validation_0-logloss:0.164836
[230]	validation_0-logloss:0.164484
[231]	validation_0-logloss:0.164122
[232]	validation_0-logloss:0.163775
[233]	validation_0-logloss:0.163426
[234]	validation_0-logloss:0.163083
[235]	validation_0-logloss:0.162737
[236]	validation_0-logloss:0.162401
[237]	validation_0-logloss:0.162079
[238]	validation_0-logloss:0.161755
[239]	validation_0-logloss:0.161428
[240]	validation_0-logloss:0.161109
[241]	validation_0-logloss:0.160787
[242]	validation_0-logloss:0.160485
[243]	validation_0-logloss:0.160167
[244]	validation_0-logloss:0.159854
[245]	validation_0-logloss:0.159558
[246]	validation_0-logloss:0.159255
[247]	validation_0-logloss:0.158955
[248]	validation_0-logloss:0.158661
[249]	validation_0-logloss:0.158375
[250]	validation_0-logloss:0.158084
[251]	validation_0-logloss:0.157801
[252]	validation_0-logloss:0.157523
[253]	validation_0-logloss:0.157248
[254]	validation_0-logloss:0.

[456]	validation_0-logloss:0.136113
[457]	validation_0-logloss:0.136083
[458]	validation_0-logloss:0.136048
[459]	validation_0-logloss:0.136011
[460]	validation_0-logloss:0.135978
[461]	validation_0-logloss:0.135946
[462]	validation_0-logloss:0.135921
[463]	validation_0-logloss:0.135889
[464]	validation_0-logloss:0.135863
[465]	validation_0-logloss:0.135836
[466]	validation_0-logloss:0.135801
[467]	validation_0-logloss:0.135778
[468]	validation_0-logloss:0.135751
[469]	validation_0-logloss:0.135726
[470]	validation_0-logloss:0.13571
[471]	validation_0-logloss:0.135689
[472]	validation_0-logloss:0.135658
[473]	validation_0-logloss:0.135632
[474]	validation_0-logloss:0.135616
[475]	validation_0-logloss:0.135598
[476]	validation_0-logloss:0.135574
[477]	validation_0-logloss:0.135553
[478]	validation_0-logloss:0.135527
[479]	validation_0-logloss:0.135507
[480]	validation_0-logloss:0.135486
[481]	validation_0-logloss:0.13546
[482]	validation_0-logloss:0.135438
[483]	validation_0-logloss:0.1

[685]	validation_0-logloss:0.133309
[686]	validation_0-logloss:0.133303
[687]	validation_0-logloss:0.1333
[688]	validation_0-logloss:0.133293
[689]	validation_0-logloss:0.133291
[690]	validation_0-logloss:0.13329
[691]	validation_0-logloss:0.133281
[692]	validation_0-logloss:0.133279
[693]	validation_0-logloss:0.133276
[694]	validation_0-logloss:0.133271
[695]	validation_0-logloss:0.13327
[696]	validation_0-logloss:0.133269
[697]	validation_0-logloss:0.133264
[698]	validation_0-logloss:0.133257
[699]	validation_0-logloss:0.133251
[700]	validation_0-logloss:0.133247
[701]	validation_0-logloss:0.133245
[702]	validation_0-logloss:0.133236
[703]	validation_0-logloss:0.133231
[704]	validation_0-logloss:0.133227
[705]	validation_0-logloss:0.133225
[706]	validation_0-logloss:0.133221
[707]	validation_0-logloss:0.133215
[708]	validation_0-logloss:0.133213
[709]	validation_0-logloss:0.133208
[710]	validation_0-logloss:0.133206
[711]	validation_0-logloss:0.133206
[712]	validation_0-logloss:0.133

[914]	validation_0-logloss:0.132613
[915]	validation_0-logloss:0.132612
[916]	validation_0-logloss:0.132609
[917]	validation_0-logloss:0.132605
[918]	validation_0-logloss:0.132606
[919]	validation_0-logloss:0.132604
[920]	validation_0-logloss:0.132604
[921]	validation_0-logloss:0.132599
[922]	validation_0-logloss:0.132595
[923]	validation_0-logloss:0.132593
[924]	validation_0-logloss:0.13259
[925]	validation_0-logloss:0.132588
[926]	validation_0-logloss:0.132588
[927]	validation_0-logloss:0.132582
[928]	validation_0-logloss:0.13258
[929]	validation_0-logloss:0.132578
[930]	validation_0-logloss:0.132577
[931]	validation_0-logloss:0.132576
[932]	validation_0-logloss:0.132573
[933]	validation_0-logloss:0.132566
[934]	validation_0-logloss:0.132565
[935]	validation_0-logloss:0.13256
[936]	validation_0-logloss:0.132559
[937]	validation_0-logloss:0.132556
[938]	validation_0-logloss:0.132552
[939]	validation_0-logloss:0.132551
[940]	validation_0-logloss:0.132544
[941]	validation_0-logloss:0.13

...

[3812]	validation_0-logloss:0.130323
[3813]	validation_0-logloss:0.130323
[3814]	validation_0-logloss:0.130322
[3815]	validation_0-logloss:0.130322
[3816]	validation_0-logloss:0.130322
[3817]	validation_0-logloss:0.130321
[3818]	validation_0-logloss:0.130321
[3819]	validation_0-logloss:0.13032
[3820]	validation_0-logloss:0.13032
[3821]	validation_0-logloss:0.13032
[3822]	validation_0-logloss:0.130319
[3823]	validation_0-logloss:0.130319
[3824]	validation_0-logloss:0.13032
[3825]	validation_0-logloss:0.130319
[3826]	validation_0-logloss:0.130319
[3827]	validation_0-logloss:0.130319
[3828]	validation_0-logloss:0.130319
[3829]	validation_0-logloss:0.130318
[3830]	validation_0-logloss:0.130318
[3831]	validation_0-logloss:0.130315
[3832]	validation_0-logloss:0.130315
[3833]	validation_0-logloss:0.130315
[3834]	validation_0-logloss:0.130315
[3835]	validation_0-logloss:0.130315
[3836]	validation_0-logloss:0.130313
[3837]	validation_0-logloss:0.130312
[3838]	validation_0-logloss:0.130313
[3839

[4035]	validation_0-logloss:0.130255
[4036]	validation_0-logloss:0.130254
[4037]	validation_0-logloss:0.130253
[4038]	validation_0-logloss:0.130253
[4039]	validation_0-logloss:0.130251
[4040]	validation_0-logloss:0.13025
[4041]	validation_0-logloss:0.13025
[4042]	validation_0-logloss:0.13025
[4043]	validation_0-logloss:0.13025
[4044]	validation_0-logloss:0.130249
[4045]	validation_0-logloss:0.13025
[4046]	validation_0-logloss:0.13025
[4047]	validation_0-logloss:0.130251
[4048]	validation_0-logloss:0.13025
[4049]	validation_0-logloss:0.13025
[4050]	validation_0-logloss:0.130249
[4051]	validation_0-logloss:0.130249
[4052]	validation_0-logloss:0.130248
[4053]	validation_0-logloss:0.130247
[4054]	validation_0-logloss:0.130247
[4055]	validation_0-logloss:0.130246
[4056]	validation_0-logloss:0.130246
[4057]	validation_0-logloss:0.130244
[4058]	validation_0-logloss:0.130244
[4059]	validation_0-logloss:0.130244
[4060]	validation_0-logloss:0.130244
[4061]	validation_0-logloss:0.130244
[4062]	va

[4257]	validation_0-logloss:0.130194
[4258]	validation_0-logloss:0.130194
[4259]	validation_0-logloss:0.130194
[4260]	validation_0-logloss:0.130194
[4261]	validation_0-logloss:0.130194
[4262]	validation_0-logloss:0.130194
[4263]	validation_0-logloss:0.130194
[4264]	validation_0-logloss:0.130194
[4265]	validation_0-logloss:0.130194
[4266]	validation_0-logloss:0.130194
[4267]	validation_0-logloss:0.130193
[4268]	validation_0-logloss:0.130193
[4269]	validation_0-logloss:0.130192
[4270]	validation_0-logloss:0.130193
[4271]	validation_0-logloss:0.130193
[4272]	validation_0-logloss:0.130193
[4273]	validation_0-logloss:0.130193
[4274]	validation_0-logloss:0.130193
[4275]	validation_0-logloss:0.130195
[4276]	validation_0-logloss:0.130194
[4277]	validation_0-logloss:0.130193
[4278]	validation_0-logloss:0.130193
[4279]	validation_0-logloss:0.130193
[4280]	validation_0-logloss:0.130193
[4281]	validation_0-logloss:0.130193
[4282]	validation_0-logloss:0.130193
[4283]	validation_0-logloss:0.130193
[

[4479]	validation_0-logloss:0.130156
[4480]	validation_0-logloss:0.130157
[4481]	validation_0-logloss:0.130155
[4482]	validation_0-logloss:0.130155
[4483]	validation_0-logloss:0.130155
[4484]	validation_0-logloss:0.130157
[4485]	validation_0-logloss:0.130157
[4486]	validation_0-logloss:0.130156
[4487]	validation_0-logloss:0.130157
[4488]	validation_0-logloss:0.130153
[4489]	validation_0-logloss:0.130153
[4490]	validation_0-logloss:0.130153
[4491]	validation_0-logloss:0.130153
[4492]	validation_0-logloss:0.130153
[4493]	validation_0-logloss:0.130153
[4494]	validation_0-logloss:0.130152
[4495]	validation_0-logloss:0.130152
[4496]	validation_0-logloss:0.130152
[4497]	validation_0-logloss:0.130152
[4498]	validation_0-logloss:0.130152
[4499]	validation_0-logloss:0.130151
[4500]	validation_0-logloss:0.130151
[4501]	validation_0-logloss:0.130151
[4502]	validation_0-logloss:0.130151
[4503]	validation_0-logloss:0.130151
[4504]	validation_0-logloss:0.130151
[4505]	validation_0-logloss:0.130151
[

FileNotFoundError: [Errno 2] No such file or directory: '../output/model/0308/xgb_churn_0.model'

In [25]:
i

0

In [27]:
import pickle
seed = 71
np.random.seed(seed) 

DATE = '0308'
utils.mkdir_p('../output/model/{}_{}/'.format(DATE,seed))
utils.mkdir_p('../output/sub/{}_{}/'.format(DATE,seed))

model
pickle.dump(model, open('../output/model/{}_{}/xgb_churn_{}.model'.format(DATE, seed, i), "wb"))

In [35]:
model.predict(test.values[:,1:])

XGBoostError: need to call fit beforehand

In [2]:
#==============================================================================
print('test')
#==============================================================================
file_name = '../output/model/xgb_feature_tuning_seed_72.model'
test = utils.load_pred_feature('test',keep_all = False, model_file_name = file_name, n_top_features = 48).fillna(-1)
sub_test = test[['msno']]
test.drop('msno',axis = 1,inplace = True) # remove msno for subsequent prediting
sub_test['is_churn'] = 0 


  0%|          | 0/20 [00:00<?, ?it/s]

test
keep top imp
True


100%|██████████| 20/20 [01:59<00:00,  5.98s/it]


test.shape:(907471, 49)



In [3]:
test.head()

,membership_duration-mean_n1,membership_duration-std_n5,days_since_the_last_expiration-mean_n1,membership_duration-min_n5,days_since_the_last_subscription-mean_n1,membership_duration-std,days_since_the_last_expiration-cumsum-std_n5,membership_duration-mean_n5,is_membership_duration_equal_to_plan_days_ratio,is_membership_duration_equal_to_plan_days_cnt_n5,...,days_since_the_last_expiration_ratio-std_n5,days_since_the_last_subscription_ratio-std_n5,days_since_the_last_expiration-cumsum-std,days_since_the_last_subscription_ratio-mean,days_since_the_last_expiration-max,days_since_the_first_subscription-std_n5,num_log_in_delta1_by14,is_auto_renew_chance,days_longer_than_plan_days-max,do_spend_more_money-mean_n1
0,31,1.303841,0,28,28,0.813575,0.0,30.200001,0.285714,1,...,0.000000,0.117975,0.000000,26.551525,0,47.757721,0,21,1,0
1,30,1.303841,0,28,31,0.787752,0.0,30.200001,0.217391,1,...,0.220759,0.143174,0.000000,34.291840,61,47.599369,-16,23,31,0
2,31,1.414214,0,28,28,10.347595,0.0,30.000000,0.181818,0,...,10.795319,9.142488,264.255463,52.029617,463,47.441544,-6,11,1,0
3,31,1.303841,0,28,28,0.870260,0.0,30.200001,0.294118,1,...,0.000000,0.200811,0.000000,25.790941,0,47.757721,0,17,1,0
4,30,1.500000,1,27,28,1.500000,1.0,29.250000,0.750000,3,...,0.127294,3.629866,1.000000,19.555555,1,29.512709,0,4,0,0


In [4]:
# load model from file
file_path1 = '../output/model/0308_71/xgb_churn_0.model'
file_path2 = '../output/model/0308_71/xgb_churn_1.model'
loaded_model1 = pickle.load(open(file_path1, "rb"))
loaded_model2 = pickle.load(open(file_path2, "rb"))
models = []
models.append(loaded_model1)
models.append(loaded_model2)

In [16]:
features = list(loaded_model1.get_booster().get_fscore().keys())

In [5]:
models = []
models.append(loaded_model1)
models.append(loaded_model2)

In [10]:
loaded_model1.predict_proba(test)[:,1].clip(min = 0.+1e-15, max = 1-1e-15)


array([ 0.01825741,  0.04357274,  0.01594887,  0.01930404,  0.09786011], dtype=float32)

In [12]:
0.+1e-15

1e-15

In [13]:
LOOP = 2
for model in models:
    sub_test['is_churn'] += model.predict_proba(test)[:,1].clip(min = 0.+1e-15, max = 1-1e-15)
sub_test['is_churn'] /= LOOP # do some simple ensemble: average of prediting result

print('Test Mean:', sub_test['is_churn'].mean())



Test Mean: 0.0715277595875


In [15]:
sub_test.head()

,msno,is_churn
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0.017137
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0.046959
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0.014721
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0.018330
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0.097995


In [18]:
# saving for submitting
DATE = '0308'
seed = 71
sub_test.to_csv('../output/sub/{}_{}/sub_test.csv'.format(DATE,seed), index = False)

